<a href="https://colab.research.google.com/github/Ducktard/Text2KnowledgeGraph/blob/dev/text3KnowledgeGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 5.2 MB/s eta 0:00:00


In [2]:
!wget https://raw.githubusercontent.com/bundestag/gesetze/refs/heads/master/h/hgb/index.md

--2025-04-25 10:49:46--  https://raw.githubusercontent.com/bundestag/gesetze/refs/heads/master/h/hgb/index.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 776684 (758K) [text/plain]
Saving to: ‘index.md’

index.md            100%[===================>] 758.48K  --.-KB/s    in 0.02s   

2025-04-25 10:49:47 (33.9 MB/s) - ‘index.md’ saved [776684/776684]



In [3]:
with open("index.md", encoding='utf-8') as file:
    hgb_text = file.read()

In [4]:
len(hgb_text)

762796

In [5]:
import groq
groq_api_key = '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'

client = groq.Groq(
    api_key=groq_api_key,
)

models = {
    "llama3-70b-8192": {
        "name": "LLaMA3-70b-Instruct",
        "tokens": 8192,
        "developer": "Meta",
    },
    "llama3-8b-8192": {
        "name": "LLaMA3-8b-Instruct",
        "tokens": 8192,
        "developer": "Meta",
    },
    "mixtral-8x7b-32768": {
        "name": "Mixtral-8x7b-Instruct-v0.1",
        "tokens": 32768,
        "developer": "Mistral",
    },
    "gemma-7b-it": {"name": "Gemma-7b-it", "tokens": 8192, "developer": "Google"},
}

In [10]:
_ontology = """
        {"Person": "Person name without any adjectives"},
        "Place",
        "Object",
        "Document",
        "Concept",
        "Organisation",
        "Event",
        "Action",
    ],
    relationships=["Relationship between Any two labeled entities"],
"""

In [6]:
def splitter(n, s):
    pieces = s.split()
    return (" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n))

In [48]:
def system_message(sourcetext) -> str:
      return (
          "You are an expert at creating Knowledge Graphs. "
          "Consider the following ontology. \n"
          f"{_ontology} \n"
          "The user will provide you with an input text delimited by ```. "
          "Extract all the entities and relationships from the user-provided text as per the given ontology. Do not use any previous knowledge about the context."
          "Remember there can be multiple direct (explicit) or implied relationships between the same pair of nodes. "
          "Be consistent with the given ontology. Use ONLY the labels and relationships mentioned in the ontology. "
          "Format your output as a json with the following schema. \n"
          "[\n"
          "   {\n"
          '       node_1: Required, an entity object with attributes: {"label": "as per the ontology", "name": "Name of the entity"},\n'
          '       node_2: Required, an entity object with attributes: {"label": "as per the ontology", "name": "Name of the entity"},\n'
          "       relationship: Describe the relationship between node_1 and node_2 as per the context, in one word.\n"
          "   },\n"
          "]\n"
          "Do not add any other comment before or after the json. Respond ONLY with a well formed json that can be directly read by a program."
          "``` \n"
          f"{sourcetext} \n"
          " ```"
      )

# TODO:  Wie gross kann splitter argument sein bevor max prompt input erreicht

In [49]:
import json
import tqdm
json_result = []
for chunk in tqdm.tqdm(splitter(500, hgb_text)):
  tempmessage = system_message(chunk)
  chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=[{"role": "user", "content": tempmessage}],
            max_tokens=2048,temperature=0.0
        )
  if len(chat_completion.choices[0].message.content) > 0:
    json_result.extend(json.loads(chat_completion.choices[0].message.content))
  if len(json_result)>10:
    break

0it [00:06, ?it/s]


JSONDecodeError: Unterminated string starting at: line 169 column 8 (char 5965)

In [57]:
str(chat_completion.choices[0].message.content)

'[\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": "Ausfertigungsdatum"},\n       "relationship": "has"\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Place", "name": "RGBl"},\n       "relationship": "published_in"\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": "Zuletzt geändert durch"},\n       "relationship": "amended_by"\n   },\n   {\n       "node_1": {"label": "Event", "name": "Zuletzt geändert durch"},\n       "node_2": {"label": "Document", "name": "Art. 14 G v. 22.12.2020 I 3256"},\n       "relationship": "is"\n   },\n   {\n       "node_1": {"label": "Document", "name": "Art. 14 G v. 22.12.2020 I 3256"},\n       "node_2": {"label": "Event", "name": "Änderung durch"},\n       "relationship": "amended_by"\n   },\n   {\n       "node_1": {"label": "Event", "name": "Än

In [45]:
chat_completion.choices[0].message.content

'[\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": " Ausfertigungsdatum"},\n       "relationship": "Handelsgesetzbuch was enacted on Ausfertigungsdatum."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Place", "name": "RGBl"},\n       "relationship": "Handelsgesetzbuch was published in RGBl."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Person", "name": "not specified"},\n       "relationship": "Handelsgesetzbuch was modified by an unknown person."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Art. 14 G v. 22.12.2020 I 3256"},\n       "node_2": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "relationship": "Art. 14 G v. 22.12.2020 I 3256 modified Handelsgesetzbuch."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Art. 7 Abs. 2 G v. 12.5.2021 I

## TODO 04.04.25 result  als networkx graph importieren

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

nx.node_link_graph(json_result)

/usr/local/lib/python3.11/dist-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


AttributeError: 'list' object has no attribute 'get'

In [ ]:
chat_completion = client.chat.completions.create(
          model='llama3-70b-8192',
          messages=[{"role": "user", "content": tempmessage}],
          max_tokens=2048
      )

In [ ]:
max_tokens=2048

In [ ]:
prompt_test= """Erstelle eine Tabelle als CSV mit der Spalte Entitaeten,  fuer einen Knowledge Graphen aus dem folgenden TExt:

Make a knowledge graph out of the following paragraphs: #### § 3

(1) Auf den Betrieb der Land- und Forstwirtschaft finden die
Vorschriften des § 1 keine Anwendung.

(2) Für ein land- oder forstwirtschaftliches Unternehmen, das nach Art
und Umfang einen in kaufmännischer Weise eingerichteten
Geschäftsbetrieb erfordert, gilt § 2 mit der Maßgabe, daß nach
Eintragung in das Handelsregister eine Löschung der Firma nur nach den
allgemeinen Vorschriften stattfindet, welche für die Löschung
kaufmännischer Firmen gelten.

(3) Ist mit dem Betrieb der Land- oder Forstwirtschaft ein Unternehmen
verbunden, das nur ein Nebengewerbe des land- oder
forstwirtschaftlichen Unternehmens darstellt, so finden auf das im
Nebengewerbe betriebene Unternehmen die Vorschriften der Absätze 1 und
2 entsprechende Anwendung."""

In [ ]:
prompt_test= """Mache eine Named Entity Recognition und gebe das ergebnis als Liste zurueck auf grundlage des folgenden TExt:

#### § 1

(1) Kaufmann im Sinne dieses Gesetzbuchs ist, wer ein Handelsgewerbe
betreibt.

(2) Handelsgewerbe ist jeder Gewerbebetrieb, es sei denn, daß das
Unternehmen nach Art oder Umfang einen in kaufmännischer Weise
eingerichteten Geschäftsbetrieb nicht erfordert.


#### § 2

Ein gewerbliches Unternehmen, dessen Gewerbebetrieb nicht schon nach §
1 Abs. 2 Handelsgewerbe ist, gilt als Handelsgewerbe im Sinne dieses
Gesetzbuchs, wenn die Firma des Unternehmens in das Handelsregister
eingetragen ist. Der Unternehmer ist berechtigt, aber nicht
verpflichtet, die Eintragung nach den für die Eintragung
kaufmännischer Firmen geltenden Vorschriften herbeizuführen. Ist die
Eintragung erfolgt, so findet eine Löschung der Firma auch auf Antrag
des Unternehmers statt, sofern nicht die Voraussetzung des § 1 Abs. 2
eingetreten ist.

#### § 3

(1) Auf den Betrieb der Land- und Forstwirtschaft finden die
Vorschriften des § 1 keine Anwendung.

(2) Für ein land- oder forstwirtschaftliches Unternehmen, das nach Art
und Umfang einen in kaufmännischer Weise eingerichteten
Geschäftsbetrieb erfordert, gilt § 2 mit der Maßgabe, daß nach
Eintragung in das Handelsregister eine Löschung der Firma nur nach den
allgemeinen Vorschriften stattfindet, welche für die Löschung
kaufmännischer Firmen gelten.

(3) Ist mit dem Betrieb der Land- oder Forstwirtschaft ein Unternehmen
verbunden, das nur ein Nebengewerbe des land- oder
forstwirtschaftlichen Unternehmens darstellt, so finden auf das im
Nebengewerbe betriebene Unternehmen die Vorschriften der Absätze 1 und
2 entsprechende Anwendung."""

In [ ]:
len(prompt_test)

1693

In [ ]:
messages=[{"role": "user", "content": prompt_test}]

In [ ]:
chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=messages,
            max_tokens=max_tokens
        )

In [ ]:
print(chat_completion.choices[0].message.content)

Here is the result of the Named Entity Recognition (NER) task on the given text:

**Entities:**

1. **Organizations:**
	* Handelsregister (2 mentions)
2. **Locations:**
	* None
3. **People:**
	* Kaufmann (1 mention)
	* Unternehmer (2 mentions)
4. **Legislation:**
	* Gesetzbuchs (2 mentions)
	* § 1 (2 mentions)
	* § 2 (2 mentions)
	* § 3 (1 mention)

Please note that the entities recognized are based on the context of the text and may not be exhaustive. Additionally, the mentions of each entity are counted separately.


In [ ]:
prompt_2 = """Du sollst einen Knowledge Graphen aus einem Text machen. Gehe dazu in folgenden SChritten vor: Schritt 1: Mache eine Named Entity Recognition und nutze das Ergebnis um im Schritt2  eine CSV TAbelle mit den folgenden Spalten: sourcenode, targetnode, relationship auf grundlage der NER und des textes zu erstellen Der zugrunde liegender TExt::
#### § 1

(1) Kaufmann im Sinne dieses Gesetzbuchs ist, wer ein Handelsgewerbe
betreibt.

(2) Handelsgewerbe ist jeder Gewerbebetrieb, es sei denn, daß das
Unternehmen nach Art oder Umfang einen in kaufmännischer Weise
eingerichteten Geschäftsbetrieb nicht erfordert.


#### § 2

Ein gewerbliches Unternehmen, dessen Gewerbebetrieb nicht schon nach §
1 Abs. 2 Handelsgewerbe ist, gilt als Handelsgewerbe im Sinne dieses
Gesetzbuchs, wenn die Firma des Unternehmens in das Handelsregister
eingetragen ist. Der Unternehmer ist berechtigt, aber nicht
verpflichtet, die Eintragung nach den für die Eintragung
kaufmännischer Firmen geltenden Vorschriften herbeizuführen. Ist die
Eintragung erfolgt, so findet eine Löschung der Firma auch auf Antrag
des Unternehmers statt, sofern nicht die Voraussetzung des § 1 Abs. 2
eingetreten ist.

#### § 3

(1) Auf den Betrieb der Land- und Forstwirtschaft finden die
Vorschriften des § 1 keine Anwendung.

(2) Für ein land- oder forstwirtschaftliches Unternehmen, das nach Art
und Umfang einen in kaufmännischer Weise eingerichteten
Geschäftsbetrieb erfordert, gilt § 2 mit der Maßgabe, daß nach
Eintragung in das Handelsregister eine Löschung der Firma nur nach den
allgemeinen Vorschriften stattfindet, welche für die Löschung
kaufmännischer Firmen gelten.

(3) Ist mit dem Betrieb der Land- oder Forstwirtschaft ein Unternehmen
verbunden, das nur ein Nebengewerbe des land- oder
forstwirtschaftlichen Unternehmens darstellt, so finden auf das im
Nebengewerbe betriebene Unternehmen die Vorschriften der Absätze 1 und
2 entsprechende Anwendung.

unter zuhilfenahme der folgenden Named Entity Recognition: """ + chat_completion.choices[0].message.content

In [ ]:
len(prompt_2)

2507

In [ ]:
messages=[{"role": "user", "content": prompt_2}]

In [ ]:
chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=messages,
            max_tokens=max_tokens
        )

In [ ]:
print(chat_completion.choices[0].message.content)

Based on the Named Entity Recognition (NER) results, I will create a CSV table with the following columns: `sourcenode`, `targetnode`, and `relationship`. Here is the resulting table:

| sourcenode | targetnode | relationship |
| --- | --- | --- |
| Kaufmann | Handelsgewerbe | is |
| Handelsgewerbe | Gewerbebetrieb | is |
| Gewerbebetrieb | Unternehmen | is_part_of |
| Unternehmen | Handelsregister | is_registered_in |
| Unternehmer | Handelsgewerbe | operates |
| Handelsgewerbe | Gesetzbuchs | is_defined_by |
| § 1 | Handelsgewerbe | defines |
| § 2 | Handelsgewerbe | defines |
| § 3 | Land- und Forstwirtschaft | excludes |
| Land- und Forstwirtschaft | Unternehmen | is_part_of |
| Unternehmen | Handelsregister | is_registered_in |
| Unternehmer | Nebengewerbe | operates |

Here's a brief explanation of each row:

1. A Kaufmann (merchant) is a type of Handelsgewerbe (commercial enterprise).
2. A Handelsgewerbe is a type of Gewerbebetrieb (business operation).
3. A Gewerbebetrieb is pa

In [ ]:
hgb_text

'---\nTitle: Handelsgesetzbuch\njurabk: HGB\nlayout: default\norigslug: hgb\nslug: hgb\n\n---\n\n# Handelsgesetzbuch (HGB)\n\nAusfertigungsdatum\n:   1897-05-10\n\nFundstelle\n:   RGBl: 1897, 219\n\nZuletzt geändert durch\n:   Art. 14 G v. 22.12.2020 I 3256\n\nÄnderung durch\n:   Art. 7 Abs. 2 G v. 12.5.2021 I 990 (Nr. 23) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 10 G v. 3.6.2021 I 1498 (Nr. 30) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 11 G v. 3.6.2021 I 1534 (Nr. 30) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 4 G v. 16.7.2021 I 3079 (Nr. 47) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 5 G v. 7.8.2021 I 3311 (Nr. 51) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 1 G v. 5.7.2021 I 3338 (Nr. 52) textlic